In [303]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dementia-patient-health-and-prescriptions-dataset/dementia_patients_health_data.csv


In [304]:
# Load the dataset
data = pd.read_csv('/kaggle/input/dementia-patient-health-and-prescriptions-dataset/dementia_patients_health_data.csv')

# Clean the data : 

# Analyze which columns contains the NaN values
print (data.isnull().sum())


Diabetic                       0
AlcoholLevel                   0
HeartRate                      0
BloodOxygenLevel               0
BodyTemperature                0
Weight                         0
MRI_Delay                      0
Prescription                 515
Dosage in mg                 515
Age                            0
Education_Level                0
Dominant_Hand                  0
Gender                         0
Family_History                 0
Smoking_Status                 0
APOE_ε4                        0
Physical_Activity              0
Depression_Status              0
Cognitive_Test_Scores          0
Medication_History             0
Nutrition_Diet                 0
Sleep_Quality                  0
Chronic_Health_Conditions    179
Dementia                       0
dtype: int64


In [305]:

# replace the missing values with the mode of each column  
obj_col_name = ["Prescription" ,"Dosage in mg" , "Chronic_Health_Conditions"]
for col in obj_col_name : 
    mode = data[col].mode()[0]
    data[col] = data[col].fillna(mode)
    
# Convert 'Depression_Status' and 'Medication_History' from yes/no to numeric
data[["Depression_Status", "Medication_History" , "Family_History"]] = data[[
    "Depression_Status", "Medication_History" , "Family_History"]].replace({"Yes": 1, "No": 0})

# Convert categorical columns to numeric 
data = pd.get_dummies(data, columns=
["Smoking_Status","Education_Level", "Prescription", "Dosage in mg" , 
 "Dominant_Hand" ,"Gender" ,"APOE_ε4","Physical_Activity" ,"Nutrition_Diet" , 
 "Sleep_Quality" , "Chronic_Health_Conditions"])

bool_cols = data.select_dtypes(include=['bool']).columns
data[bool_cols] = data[bool_cols].astype(int)



/tmp/ipykernel_36/58716508.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  "Depression_Status", "Medication_History" , "Family_History"]].replace({"Yes": 1, "No": 0})


In [306]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


X = data.drop('Dementia' , axis = 1).values
Y = (data ['Dementia'].values).reshape(-1 , 1)
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size = 0.3 , random_state = 42 ,shuffle = True)
model = LogisticRegression()
model.fit(X_train , Y_train)

# Predict probabilities
Y_pred = model.predict(X_test)


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [307]:
print (Y_pred)
# Assess the model 
from sklearn.metrics import accuracy_score
# Calculate the accuracy 
accuracy = accuracy_score(Y_test , Y_pred)
print(accuracy)



[0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1
 1 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1
 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 1
 1 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0
 0 0 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1
 1 0 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0
 1 0 0 1 0 1 1 1 0 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 0
 1 1 0 1]
0.9966666666666667
